# NumPy高级应用

## 目录
+ ndarray对象的内部机理
	+ NumPy数据类型体系
    
    
+ 高级数组操作
	+ 数组重塑
	+ C和Fortran顺序
	+ 数组的合并和拆分
		+ 堆叠辅助类
	+ 元素的重复操作：tile和repeat
	+ 花式索引的等价函数：take和put
    
    
+ 广播
	+ 沿其他轴向广播
	+ 通过广播设置数组的值
    
    
+ ufunc高级应用
	+ Ufunc实例方法
	+ 自定义ufuncs
		+ numpy.frompyfunc
		+ numpy.vectorize
        
        
+ 结构化和记录式数组
	+ 嵌套dtype和多维字段
	+ 为什么要用结构化数组
	+ 结构化数组操作：numpy.lib.recfunctions
    
    
+ 更多有关排序的话题
	+ 间接排序：argsort和lexsort
		+ argsort
		+ lexsort
	+ 其他排序算法
	+ numpy.searchsorted：在有序数组中查找元素
    
    
+ NumPy的matrix类


+ 高级数组输入输出
	+ 内存映像文件
	+ HDF5及其他数组存储方式
    
    
+ 性能建议
	+ 连续内存的重要性
	+ 其他加速手段：Cython, f2py, C

In [ ]:
from numpy.random import randn
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pandas as pd
import sys

## ndarray对象的内部机理

+ ndarray的内部结构
    + 一个指向数组（一个系统内存块）的指针
    + 数据类型或dtype
    + 一个表示数组形状（shape）的元组
    + 一个跨度元组（stride），其中的整数指的是为了前进到当前维度下一个元素需要“跨过”的字节数

### NumPy数据类型体系
+ generic
    + number
        + integer
            + unsigned int
            + signed int
        + inexact
            + floating
            + complex
    + character
        + string_
        + unicode_
    + bool_
    + object_  
    
    
有些dtype的名称后面带有下划线，这是为了避免NumPy类型和Python类型之间的变量名冲突

In [ ]:
ints = np.ones(10, dtype=np.uint16)
floats = np.ones(10, dtype=np.float32)
np.issubdtype(ints.dtype, np.integer) # 检查ints的dtype是否是np.integer的子类
np.issubdtype(floats.dtype, np.floating) # 检查floats的dtype是否是np.floating的子类

In [ ]:
np.float64.mro() # 调用dtype的mro方法即可查看其所有的父类

## 高级数组操作

### 数组重塑

In [ ]:
arr = np.arange(8)
arr.reshape((4, 2)) # 向reshape方法传入一个表示新形状的元组

In [ ]:
arr.reshape((4, 2)).reshape((2, 4)) # 多维数组的重塑

In [ ]:
arr = np.arange(15)
arr.reshape((5, -1)) # 作为参数的形状的其中一维可以是-1，表示该维度的大小由数据本身推断而来

In [ ]:
other_arr = np.ones((3, 5))
arr.reshape(other_arr.shape) # 数组的shape属性是一个元组，因此也可以被传入reshape

In [ ]:
arr = np.arange(15).reshape((5, 3))
arr.ravel() # 数组扁平化（flattering）或散开（raveling），ravel不会产生源数据的副本

In [ ]:
arr.flatten() # flatten方法与ravel方法类似，只是总是返回数据的副本

### C和Fortran顺序
默认情况下，NumPy数组是按行优先顺序创建的  
行和列优先顺序分别称为C和Fortran顺序

+ C/行优先顺序，参数'C'  
先经过更高的维度（例如，轴1会先于轴0被处理）
+ Fortran/列优先顺序，参数'F'  
后经过更高的维度（例如，轴0会先于轴1被处理）

In [ ]:
arr = np.arange(12).reshape((3, 4))
arr.ravel()
arr.ravel('F') # reshape和ravel函数，可以接受表示数组数据存放顺序的order参数'C'或'F'

### 数组的合并和拆分
+ concatenate  
最一般化的连接，沿一条轴连接一组数组
+ vstack、row_stack  
以面向行的方式对数组进行堆叠（沿轴0）
+ hstack  
以面向列的方式对数组进行堆叠（沿轴1）
+ column_stack  
类似于hstack，但是会先将一维数组转换为二维列向量
+ dstack  
以面向“深度”的方式对数组进行堆叠（沿轴2）
+ split  
沿指定轴在指定的位置拆分数组
+ hsplit、vsplit、dsplit  
split的便捷化函数，分别沿轴0、轴1、轴2进行拆分

In [ ]:
arr1 = np.array([[1, 2, 3], [4, 5, 6]])
arr2 = np.array([[7, 8, 9], [10, 11, 12]])
np.concatenate([arr1, arr2], axis=0) # 按行纵向连接
np.concatenate([arr1, arr2], axis=1) # 按列横向连接

In [ ]:
np.vstack((arr1, arr2)) # vstack方法，按行纵向连接
np.hstack((arr1, arr2)) # hstack方法，按列横向连接

In [ ]:
from numpy.random import randn
arr = randn(5, 2)
first, second, third = np.split(arr, [1, 3]) # split用于将一个数组沿指定轴拆分为多个数组
first
second
third

#### 堆叠辅助类

In [ ]:
# r_和c_的具体功能参考其文档
arr = np.arange(6)
arr1 = arr.reshape((3, 2))
arr2 = randn(3, 2)
np.r_[arr1, arr2] # r_类似于vstack
np.c_[np.r_[arr1, arr2], arr] # c_类似于hstack

In [ ]:
np.c_[1:6, -10:-5] # 将切片组装出数组

### 元素的重复操作：tile和repeat

In [ ]:
arr = np.arange(3)
arr.repeat(3) # 传入一个整数，repeat会将数组中的各个元素重复一定次数，从而产生一个更大的数组

In [ ]:
arr.repeat([2, 3, 4]) # 传入一组整数，则各元素重复不同的次数

In [ ]:
arr = randn(2, 2)
arr.repeat(2, axis=0) # 对于多维数组，可以让元素沿指定轴重复
arr.repeat(2) # 如果没有设置轴向，则数组会被扁平化

In [ ]:
arr.repeat([2, 3], axis=0) # 对多维数组，也可以传入一组整数，使各切片重复不同的次数
arr.repeat([2, 3], axis=1)

In [ ]:
np.tile(arr, 2) # tile沿指定轴向堆叠数组的副本，如同“铺瓷砖”，第二个参数是瓷砖的数量，对于标量，瓷砖是水平铺设的

In [ ]:
np.tile(arr, (2, 1)) # 第二个参数也可以是表示“铺设”布局的元组
np.tile(arr, (3, 2))

### 花式索引的等价函数：take和put

In [ ]:
arr = np.arange(10) * 100
inds = [7, 1, 2, 6]
arr[inds]

In [ ]:
arr.take(inds) # take可以获取单个轴向上的选区
arr.put(inds, 42) # put可以设置单个轴向上的选区
arr.put(inds, [40, 41, 42, 43])

In [ ]:
inds = [2, 0, 2, 1]
arr = randn(2, 4)
arr.take(inds, axis=1) # 在其他轴上使用take，传入axis参数；put不接受axis参数，只能够在数组的扁平化版本（一维，C顺序）上进行索引

## 广播
广播（broadcasting）指的是不同形状的数组之间的算术运算的执行方式

#### 广播的原则
如果两个数组的后缘维度（trailing dimension，即从末尾开始算起的维度）的轴长度相符或其中一个的长度为1，则认为它们是广播兼容的。广播会在缺失和（或）长度为1的维度上进行

In [ ]:
arr = np.arange(5)
arr * 4 # 在乘法运算中，标量4被广播到了其他所有元素上

In [ ]:
arr = randn(4, 3)
arr.mean(0)
demeaned = arr - arr.mean(0) # 在0轴上广播
demeaned.mean(0) # 检验离差的均值为0

In [ ]:
row_means = arr.mean(1)
row_means.reshape((4, 1)) # 行平均值的形状变成(4, 1)而不是(4,)，因为不能降维，且广播维的长度必须为1
demeaned = arr - row_means.reshape((4, 1)) # 在1轴上广播
demeaned.mean(1) # 检验离差的均值为0

### 沿其他轴向广播
NumPy数组提供了一种通过索引机制插入轴的特殊语法，利用np.newaxis属性以及“全”切片来插入新轴（将较小数组的维度提升到与大数组相同）

In [ ]:
arr = np.zeros((4, 4))
arr_3d = arr[:, np.newaxis, :] # 利用np.newaxis属性以及“全”切片来插入新轴，相当于.reshape((4, 1, 4))
arr_3d.shape

In [ ]:
arr_1d = np.random.normal(size=3)
arr_1d[:, np.newaxis] # 相当于.reshape((3, 1))
arr_1d[np.newaxis, :] # 相当于.reshape((1, 3))

In [ ]:
arr = randn(3, 4, 5)
depth_means = arr.mean(2) # 沿2轴求均值
demeaned = arr - depth_means[:, :, np.newaxis] # 沿2轴求离差
demeaned.mean(2) # 检验离差的均值为0

In [ ]:
# 自定义对指定轴求离差的函数
def demean_axis(arr, axis=0):
    means = arr.mean(axis)

    # 下面这些一般化的东西类似于N维的[:, :, np.newaxis]
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[indexer]

### 通过广播设置数组的值
算术运算所遵循的广播原则同样也适用于通过索引机制设置数组值的操作

In [ ]:
arr = np.zeros((4, 3))
arr[:] = 5

In [ ]:
col = np.array([1.28, -0.42, 0.44, 1.6])
arr[:] = col[:, np.newaxis] # 用一个一维数组来设置目标数组的各列，按列
arr[:2] = [[-1.37], [0.509]] # 设置目标数组前两行，按行

## ufunc高级应用

### ufunc实例方法
NumPy的各个二元ufunc都有一些用于执行特定矢量化运算的特殊方法
+ reduce(x)  
通过连续执行原始运算的方式对值进行聚合
+ accumulate(x)  
聚合值，保留所有局部聚合结果
+ reduceat(x, bins)  
“局部”约简（也就是groupby）。约简数据的各个切片以产生聚合型数组
+ outer(x, y)  
对x和y中的每对元素应用原始运算。结果数组的形状为x.shape + y.shape

In [ ]:
arr = np.arange(10)
np.add.reduce(arr) # reduce接受一个数组参数，并通过一系列的二元运算对其值进行聚合（可指明轴向）
arr.sum() # add.reduce和sum方法时等价的

In [ ]:
np.random.seed(12346)
arr = randn(5, 5)
arr[::2].sort(1) # 对部分行排序
arr[:, :-1] < arr[:, 1:]
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis=1) # logical_and.reduce和all方法是等价的

In [ ]:
arr = np.arange(15).reshape((3, 5))
np.add.accumulate(arr, axis=1) # add.accumulate跟cumsum是等价的

In [ ]:
arr = np.arange(3).repeat([1, 2, 2])
np.multiply.outer(arr, np.arange(5)) # outer用于计算两个数组的叉积（外积）

In [ ]:
result = np.subtract.outer(randn(3, 4), randn(5))
result.shape # outer输出结果的维度是两个输入数据的维度之和

In [ ]:
arr = np.arange(10)
np.add.reduceat(arr, [0, 5, 8]) # 在arr[:5]、arr[5:8]、arr[8:]上执行约简，本例中是求和

In [ ]:
arr = np.multiply.outer(np.arange(4), np.arange(5))
np.add.reduceat(arr, [0, 2, 4], axis=1) # 传入axis参数，在列上分组

### 自定义ufuncs

#### numpy.frompyfunc
接受一个Python函数以及两个分别表示输入输出参数数量的整数，用frompyfunc创建的函数总是返回Python对象数组

In [ ]:
def add_elements(x, y):
    return x + y
add_them = np.frompyfunc(add_elements, 2, 1) # 能够实现元素级加法的简单函数
add_them(np.arange(8), np.arange(8))

#### numpy.vectorize
功能没有numpy.frompyfunc那么强大，但在类型推断方面更加智能

In [ ]:
add_them = np.vectorize(add_elements, otypes=[np.float64])
add_them(np.arange(8), np.arange(8))

In [ ]:
arr = randn(10000)
%timeit add_them(arr, arr)
%timeit np.add(arr, arr) # 自定义ufunc型函数速度非常慢，因为计算每个元素时都要执行一次Python函数调用

## 结构化和记录式数组
结构化数组是一种特殊的ndarray，其中的各个元素可以被看做C语言中的结构体（struct，这就是“结构化”的由来）或SQL表中带有多个命名字段的行

In [ ]:
dtype = [('x', np.float64), ('y', np.int32)] # 'x'字段和'y'字段，dtype为元组列表，格式为(field_name, field_data_type)
sarr = np.array([(1.5, 6), (np.pi, -2)], dtype=dtype) # (1,5, 6)、(np.pi, -2)分别为记录，(1.5, np.pi)为'x'字段、(6, -2)为'y'字段

In [ ]:
sarr.dtype.names # 字段名保存在dtype.names属性中

In [ ]:
sarr[0]
sarr[0]['y']

In [ ]:
sarr['x'] # 访问结构化数组的某个字段时，返回的是该数据的视图，所以不会发生数据复制

### 嵌套dtype和多维字段

In [ ]:
dtype = [('x', np.int64, 3), ('y', np.int32)] # 设置一个形状（可以是一个整数或一个元组），本例'x'字段有3个元素，'y'字段默认1个元素
arr = np.zeros(4, dtype=dtype)

In [ ]:
arr[0]['x'] # 各个记录的'x'字段是长度为3的数组

In [ ]:
arr['x'] # 访问'x'字段得到一个(4, 3)的二维数组

In [ ]:
dtype = [('x', [('a', 'f8'), ('b', 'f4')]), ('y', np.int32)] # 嵌套dtype结构
data = np.array([((1, 2), 5), ((3, 4), 6)], dtype=dtype)
data['x']
data['y']
data['x']['a'] # 可变形状的字段和嵌套记录，与pandas的DataFrame的分层索引机制类似

### 为什么要用结构化数组
+ 结构化数组能够提供非常快速高效的磁盘数据读写（包括内存映像）、网络传输等功能
+ 将数据文件写成定长记录字节流，这是C和C++代码中常见的数据序列化手段。只要知道文件的格式（记录的大小、元素的顺序、字节数以及数据类型等），就可以用np.fromfile将数据读入内存

### 结构化数组操作：numpy.lib.recfunctions
NumPy模块numpy.lib.recfunctions中有一些用于增删字段或执行基本连接运算的工具。对于这些工具，我们需要记住的是：一般都需要创建一个新数组以便对dtype进行修改（比如添加或删除一列）。

## 更多有关排序的话题

In [ ]:
arr = randn(6)
arr.sort() # 跟Python内置的列表一样，ndarray的sort实例方法也是就地排序

In [ ]:
arr = randn(3, 5)
arr[:, 0].sort()  # 对第1列按值排序

In [ ]:
arr = randn(5)
np.sort(arr) # np.sort会为原数组创建一个已排序副本

In [ ]:
arr = randn(3, 5)
arr.sort(axis=1) # 默认按行排序，接受一个axis参数，以便沿指定轴向对各块数据进行单独排序

In [ ]:
arr[:, ::-1] # sort不可设置为降序，用切片的方式实现降序（切片产生视图，而非副本）

### 间接排序：argsort和lexsort

#### argsort

In [ ]:
values = np.array([5, 0, 1, 3, 2])
indexer = values.argsort() # 生成索引器，其中的索引值说明了数据在新顺序下的位置
values[indexer] # 实现升序排序

In [ ]:
arr = randn(3, 5)
arr[0] = values
arr[:, arr[0].argsort()] # 根据数组的第一行对其进行排序

#### lexsort
lexsort跟argsort差不多，但它可以一次性对多个键数组执行间接排序（字典序）

In [ ]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])
sorter = np.lexsort((first_name, last_name)) # 键的应用顺序是从最后一个传入的键算起的
zip(last_name[sorter], first_name[sorter])

### 其他排序算法
稳定的（stable）排序算法会保持等价元素的相对位置。对于相对位置具有实际意义的那些间接排序而言，这一点非常重要。 

|  kind    |  速度   |   稳定性    | 工作空间    |    最坏的情况   |
| ------------- |:-------------| :-----| :-----| :-----|
| 'quicksort'     | 1 | 否 |  0  | O(n2) |
| 'mergesort'     | 2 | 是 |  n/2  |O(nlogn) |
| 'heapsort'     | 3 | 否 |  0  |  O(nlogn)  |

In [ ]:
values = np.array(['2:first', '2:second', '1:first', '1:second', '1:third'])
key = np.array([2, 2, 1, 1, 1])
indexer = key.argsort(kind='mergesort') # 参数kind，选择mergesort（合并排序），是唯一的稳定排序
values.take(indexer)

### numpy.searchsorted：在有序数组中查找元素
searchsorted是一个在有序数组上执行二分查找的数组方法，只要将值插入到它返回的那个位置就能维持数组的有序性

In [ ]:
arr = np.array([0, 1, 7, 12, 15])
arr.searchsorted(9)

In [ ]:
arr.searchsorted([0, 8, 11, 16]) # 传入一组值就能得到一组索引

In [ ]:
arr = np.array([0, 0, 0, 1, 1, 1, 1])
arr.searchsorted([0, 1]) # 默认返回相等值组的左侧索引，因而对于元素0，默认会返回0
arr.searchsorted([0, 1], side='right') # 设置side参数，返回相等值组的右侧索引

In [ ]:
data = np.floor(np.random.uniform(0, 10000, size=50))
bins = np.array([0, 100, 1000, 5000, 10000]) # 将数据数组依照“面元边界”数组拆分开

In [ ]:
labels = bins.searchsorted(data) # 得到各数据点所属区间的编号

In [ ]:
Series(data).groupby(labels).mean() # 利用pandas的groupby使用该结果，可非常轻松地对原数据集进行拆分

In [ ]:
np.digitize(data, bins) # np.digitize函数也可以用于计算面元编号

## NumPy的matrix类
NumPy提供了一个matrix类，其索引行为更像MATLAB：单行或列会以二维形式返回，且使用星号（*）的乘法直接就是矩阵乘法  
由于numpy.matrix类的应用面较窄，一般不建议用numpy.matrix替代ndarray。对于个别带有大量线性代数运算的函数，可以将函数参数转换为matrix类型，然后在返回之前用np.asarray（不会复制任何数据）将其转换回正规的ndarray

In [ ]:
X =  np.array([[ 8.82768214,  3.82222409, -1.14276475,  2.04411587],
               [ 3.82222409,  6.75272284,  0.83909108,  2.08293758],
               [-1.14276475,  0.83909108,  5.01690521,  0.79573241],
               [ 2.04411587,  2.08293758,  0.79573241,  6.24095859]])
X[:, 0]  # 一维数组
y = X[:, :1]  # 切片操作可产生二维结果

In [ ]:
np.dot(y.T, np.dot(X, y)) # 矩阵乘法

In [ ]:
Xm = np.matrix(X) # np.asarray可以将matrix转换回ndarray
ym = Xm[:, 0]
ym.T * Xm * ym # 上面的代码用numpy.matrix来编写的形式

In [ ]:
Xm.I * X # 属性I，功能是返回矩阵的逆

## 高级数组输入输出

### 内存映像文件
内存映像文件是一种将磁盘上的非常大的二进制数据文件当做内存中的数组进行处理的方式  
NumPy实现了一个类似于ndarray的memmap对象，它允许将大文件分成小段进行读写，而不是一次性将整个数组读入内存  
memmap也拥有跟普通数组一样的方法，因此基本上只要是能用于ndarray的算法就也能用于memmap

In [ ]:
# 使用np.memmap函数，并传入一个文件路径、数据类型、形状以及文件模式，即可创建一个新的memmap
mmap = np.memmap('mymmap', dtype='float64', mode='w+', shape=(10000, 10000))

In [ ]:
section = mmap[:5] # 对memmap切片将会返回磁盘上的数据的视图

In [ ]:
section[:] = np.random.randn(5, 10000)
mmap.flush()
mmap
del mmap # 如果将数据赋值给这些视图：数据会被先缓存在内存中，调用flush即可将其写入磁盘

In [ ]:
# 只要某个内存映像超出了作用域，它就会被垃圾回收器回收，之前对其所做的任何修改都会被写入磁盘
mmap = np.memmap('mymmap', dtype='float64', shape=(10000, 10000)) # 打开一个已经存在的内存映像时，仍然需要指明数据类型和形状

In [ ]:
%xdel mmap
!rm mymmap

### HDF5及其他数组存储方式
PyTables和h5py这两个Python项目可以将NumPy的数组数据存储为高效且可压缩的HDF5格式（HDF意思是“层次化数据格式”）。你可以安全地将好几百GB甚至TB的数据存储为HDF5格式  
PyTables提供了一些用于结构化数组的高级查询功能，而且还能添加列索引以提升查询速度。这跟关系型数据库所提供的表索引功能非常类似

## 性能建议

### 连续内存的重要性
数组的内存布局可以对计算速度造成极大的影响。运算过程中访问连续内存块（例如，对以C顺序存储的数组的行求和）一般是最快的  
一个数组的内存布局是连续的，就是说元素是以它们在数组中出现的顺序（即Fortran型（列优先）或C型（行优先））存储在内存中的。默认情况下，NumPy数组是以C型连续的方式创建的

In [ ]:
arr_c = np.ones((1000, 1000), order='C')
arr_f = np.ones((1000, 1000), order='F')
arr_c.flags
arr_f.flags # 通过ndarray的flags属性，可以查看相关信息
arr_f.flags.f_contiguous # 查看是否F型连续

In [ ]:
%timeit arr_c.sum(1)
%timeit arr_f.sum(1) # 对两个数组的行求和，arr_c会比arr_f块，因为arr_c的行在内存中是连续的

In [ ]:
arr_f.copy('C').flags # 如果数组的内存顺序不符合要求，使用copy并传入'C'或'F'即可解决该问题

In [ ]:
arr_c[:50].flags.contiguous
arr_c[:, :50].flags # 在构造数组的视图时，其结果不一定是连续的

In [ ]:
%xdel arr_c
%xdel arr_f
%cd ..

## 其他加速手段：Cython, f2py, C
可以将Cython看成是带有静态类型并能嵌入C函数的Python  
其他有关NumPy的高性能代码编写手段还有f2py（FORTRAN 77和90的包装器生成器）以及利用纯C语言编写Python扩展

```cython
from numpy cimport ndarray, float64_t

def sum_elements(ndarray[float64_t] arr):
    cdef Py_ssize_t i, n = len(arr)
    cdef float64_t result = 0

    for i in range(n):
        result += arr[i]

    return result
```

Cython处理这段代码时，先将其翻译为C代码，然后编译这些C代码并创建一个Python扩展  
Cython是一种诱人的高性能计算方式，因为编写Cython代码只比编写纯Python代码多花一点时间，而且还能跟NumPy紧密结合  
一般的工作流程是：得到能在Python中运行的算法，然后将其翻译为Cython（只需添加类型定义并完成一些其他必要的工作即可）